In [266]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from pathlib import Path
from langchain.docstore.document import Document
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langgraph.graph import Graph
import torch 
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
load_dotenv()

True

In [267]:
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


In [268]:
class Utf8TextLoader(TextLoader):
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        with open(self.file_path, 'r', encoding='utf-8') as f:
            return f.read()


def read_text_file(file_path):
    return Path(file_path).read_text(encoding='utf-8')


def create_documents_from_files(file_paths):
    documents = []
    for file_path in file_paths:
        content = read_text_file(file_path)
        documents.append(Document(page_content=content))
    return documents

In [269]:
file_paths = list(Path('./source').glob("*.txt"))
docs = create_documents_from_files(file_paths)

In [270]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

In [271]:
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]
print(doc_strings)

['El deporte es una actividad física que desempeña un papel crucial en la vida de las personas en todo el mundo. No solo es una fuente de entretenimiento y competencia, sino que también promueve la salud física y mental, fomenta el trabajo en equipo y contribuye al desarrollo de valores sociales y culturales. La diversidad de disciplinas deportivas, desde el fútbol y el baloncesto hasta el atletismo', 'y el baloncesto hasta el atletismo y la natación, ofrece opciones para todos los gustos y habilidades, haciendo del deporte una parte integral de la sociedad moderna.', 'Uno de los deportes más populares a nivel global es el fútbol. Conocido como "el deporte rey", el fútbol se juega y se sigue fervientemente en todos los continentes. La Copa Mundial de la FIFA, celebrada cada cuatro años, es uno de los eventos deportivos más vistos en el planeta, uniendo a millones de aficionados en una celebración de habilidad, estrategia y pasión. Las ligas de fútbol de todo el', 'pasión. Las ligas de 

In [272]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [273]:
print(f"Using device: {device}")

Using device: cpu


In [274]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
embeddings.__dict__

d:\Proyecto_holagrama_IA\LanGraph\v2email\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'client': SentenceTransformer(
   (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
   (2): Normalize()
 ),
 'model_name': 'sentence-transformers/all-mpnet-base-v2',
 'cache_folder': None,
 'model_kwargs': {'device': 'cpu'},
 'encode_kwargs': {'normalize_embeddings': True},
 'multi_process': False,
 'show_progress': False}

In [275]:
db = Chroma.from_documents(new_docs, embeddings)
db

In [276]:
retriever = db.as_retriever(search_kwargs={"k": 4})
retriever.__dict__

{'name': None,
 'tags': ['Chroma', 'HuggingFaceEmbeddings'],
 'metadata': None,
 'vectorstore': <langchain_community.vectorstores.chroma.Chroma at 0x17eef41d0c0>,
 'search_type': 'similarity',
 'search_kwargs': {'k': 4}}

In [277]:
query = "Cuentame sobre la cultura japonesa"
docs = retriever.invoke(query)
print(docs)

[Document(page_content='La cultura japonesa es un fascinante contraste entre la tradición y la modernidad. Por un lado, el país es famoso por sus antiguas prácticas y costumbres, como el arte del té, la ceremonia del kimono, y los jardines zen. Lugares emblemáticos como el Templo Senso-ji en Tokio, el Santuario Fushimi Inari-taisha en Kioto y el Castillo de Himeji son testamentos de su herencia histórica y'), Document(page_content='La cultura japonesa es un fascinante contraste entre la tradición y la modernidad. Por un lado, el país es famoso por sus antiguas prácticas y costumbres, como el arte del té, la ceremonia del kimono, y los jardines zen. Lugares emblemáticos como el Templo Senso-ji en Tokio, el Santuario Fushimi Inari-taisha en Kioto y el Castillo de Himeji son testamentos de su herencia histórica y'), Document(page_content='La cultura japonesa es un fascinante contraste entre la tradición y la modernidad. Por un lado, el país es famoso por sus antiguas prácticas y costumbre

En extensión .py funciona bien y da el formato bien

In [278]:
AgentState = {}
AgentState["messages"] = []

In [279]:
AgentState

{'messages': []}

In [280]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192", streaming=False)

In [281]:
def function_1(state):
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    
    complete_query = "Su tarea es proporcionar sólo el tema basado en la consulta del usuario. \
        Sólo se emite el tema entre: [Japan , Sports]. No incluya el razonamiento. A continuación se muestra la consulta del usuario: " + question
    response = llm.invoke(complete_query)
    state['messages'].append(response.content) # appending LLM call response to the AgentState
    return state

In [282]:
def function_2(state):
    messages = state['messages']
    question = messages[0] 

    template = """Responda a la pregunta basándose únicamente en el siguiente contexto:
    {context}

    Pregunta: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return result

In [283]:
workflow = Graph()

workflow.add_node("Agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('Agent', 'tool')

workflow.set_entry_point("Agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [284]:
inputs = {"messages": ["Háblame del crecimiento industrial de Japón"]}
app.invoke(inputs)

'Según el texto, Japón es un líder mundial en tecnología e innovación. Ciudades como Tokio y Osaka son centros neurálgicos de avances tecnológicos, y son hogar de gigantes de la industria como Sony, Toyota y Nintendo. Esto sugiere que Japón ha experimentado un crecimiento industrial significativo, especialmente en sectores como la tecnología y la manufactura.'

In [285]:
inputs = {"messages": ["¿El deporte reduce el riesgo de enfermedades crónicas como la diabetes?"]}
app.invoke(inputs)

'No hay información en el contexto proporcionado que permita responder a esta pregunta. El contexto solo habla sobre el tenis y su crecimiento en todo el mundo, pero no menciona nada sobre la relación entre el deporte y el riesgo de enfermedades crónicas como la diabetes.'

In [286]:
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Salida del Nodo '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Salida del Nodo 'Agent':
---
{'messages': ['¿El deporte reduce el riesgo de enfermedades crónicas como la diabetes?', 'Sports', 'Sports']}

---

Salida del Nodo 'tool':
---
No hay información en el contexto proporcionado que permita responder a esta pregunta. El contexto solo habla sobre el tenis y su crecimiento en todo el mundo, pero no menciona nada sobre la relación entre el deporte y el riesgo de enfermedades crónicas como la diabetes.

---



# INICIO 26

In [287]:
# Definir el modelo Pydantic
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [288]:
class TopicSelectionParser(BaseModel):
    Topic: str = Field(description='Selected Topic')
    #Reasoning: str = Field(description='Reasoning behind topic selection')

# Crear el parser
parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)
parser

PydanticOutputParser(pydantic_object=<class '__main__.TopicSelectionParser'>)

In [289]:
def function_1(state):
    print('-> Calling Agent ->')
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    
    templete = """ Su tarea es proporcionar sólo el tema basado en la consulta del usuario.  
        Sólo se emite el tema entre: [Japan , Sports , Not Related]. No incluya el razonamiento. A continuación se muestra la consulta del usuario:  {question}
        {format_instructions} """
    prompt = PromptTemplate(template=templete,
                                    input_variables=[question],
                                    partial_variables={
                                        "format_instructions" : parser.get_format_instructions()}
                                    )
    chain = prompt | llm | parser

    response = chain.invoke({"question":question })

    print(response)

    return {"messages": [response.Topic]}


In [290]:
def function_2(state):
    print('-> Calling RAG ->')
    messages = state['messages']
    question = messages[0] ## Fetching the user question

    template = """Responda a la pregunta basándose únicamente en el siguiente contexto:
    {context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return  {"messages": [result]}

In [291]:
def function_3(state):
    print('-> Calling LLM ->')

    messages = state['messages']
    question = messages[0] ## Fetching the user question

    # Normal LLM call
    complete_query = "Responde a la siguiente pregunta con tus conocimientos del mundo real. Pregunta del usuario: " + question
    response = llm.invoke(complete_query)
    return {"messages": [response.content]}

In [292]:
def router(state):
    print('-> Router ->')
    
    messages = state["messages"]
    last_message = messages[-1]
    print(last_message)
    if 'Japan' in last_message or 'Sports' in last_message:
        return 'RAG Call'
    else:
        return 'LLM Call'


In [293]:
from langgraph.graph import StateGraph,END

graph = StateGraph(AgentState) ### StateGraph with AgentState

graph.add_node("agent", function_1)
graph.add_node("RAG", function_2)
graph.add_node("LLM", function_3)

graph.set_entry_point("agent")


# conditional edges are controlled by our router
graph.add_conditional_edges(
    "agent",  # where in graph to start
    router,  # function to determine which node is called
    {
        "RAG Call": "RAG",
        "LLM Call": "LLM",
    }
)

graph.add_edge("RAG", END)
graph.add_edge("LLM", END)

app = graph.compile()

In [294]:
inputs = {"messages": ["Hablame de la cultura de Japón"]}
out = app.invoke(inputs)

-> Calling Agent ->


Topic='Japan'
-> Router ->
Japan
-> Calling RAG ->


In [295]:
out['messages']

['Hablame de la cultura de Japón',
 'Japan',
 'Según el texto, la cultura de Japón es una nación de contrastes y armonía, donde la antigua cultura y las tradiciones coexisten con la innovación y la modernidad. Japón ha dejado una huella indeleble en el mundo a través de su arte, tecnología, gastronomía y filosofía.']

In [296]:
out['messages'][-1]

'Según el texto, la cultura de Japón es una nación de contrastes y armonía, donde la antigua cultura y las tradiciones coexisten con la innovación y la modernidad. Japón ha dejado una huella indeleble en el mundo a través de su arte, tecnología, gastronomía y filosofía.'

In [297]:
inputs = {"messages": ["¿Cuales son las mejoras de salud que ofrece el deporte?"]}
out = app.invoke(inputs)
print(out)

-> Calling Agent ->
Topic='Sports'
-> Router ->
Sports
-> Calling RAG ->
{'messages': ['¿Cuales son las mejoras de salud que ofrece el deporte?', 'Sports', 'Basándome en el contexto proporcionado, no hay información explícita sobre las mejoras de salud que ofrece el deporte. El texto solo menciona que el deporte (en este caso, el tenis) inspira a jóvenes tenistas y contribuye al crecimiento del tenis en todo el mundo, pero no proporciona información sobre los beneficios para la salud.']}


In [298]:
inputs = {"messages": ["Hablame de las jirafas"]}
out = app.invoke(inputs)
print(out)

-> Calling Agent ->
Topic='Not Related'
-> Router ->
Not Related
-> Calling LLM ->
{'messages': ['Hablame de las jirafas', 'Not Related', 'Las jirafas! Son uno de los animales más fascinantes y únicos del reino animal. Aquí te presento algunos datos interesantes sobre estas criaturas increíbles:\n\n**Características físicas**\n\n* Las jirafas son los mamíferos terrestres más altos del mundo, con una altura promedio de 4,8 a 5,5 metros (16 a 18 pies) desde la base de los cascos hasta la parte superior de la cabeza.\n* Pueden pesar entre 830 y 1.930 kilos (1.830 a 4.250 libras).\n* Tienen un cuerpo largo y delgado, con patas largas y una cabeza pequeña.\n* Su característica más destacada es su cuello largo, que puede medir hasta 2 metros (6,6 pies) de largo.\n\n**Distribución y hábitat**\n\n* Las jirafas se encuentran en África, principalmente en la sabana y los bosques abiertos de la región subsahariana.\n* Se distribuyen en una amplia área que abarca desde Sudáfrica hasta Etiopía, y de